<a id='section-zero'></a>

# Regressão Linear

**Objetivos**: Aprender o que é Regressão Linear e introduzir intuições sobre o Método do Gradiente e o Método do Gradiente Estocástico assim como os problemas de regressão de aprendizagem de máquina. Apresentar a biblioteca `SciKit-Learn`.

## Definição - Regressão Linear

> Uma regressão linear faz uma predição simplesmente computando uma soma ponderada dos atributos (*features*), mais uma constante chamada viés (*bias*), também chamado de constante (*intercept*).

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/reg-linear.png?raw=1" alt="reg-linear" style="width: 400px;"/>

$$ \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n$$

$\hat{y}$ - valor previsto

$\theta$ - parâmetro do modelo

$n$ - número de atributos (*features*)

$x_i$ - o valor do *inésimo* atributo (*feature*)

### Exemplo

$$\mathrm{preço~de~residência} = 4500 + 1000\times \mathrm{quartos} + 120 \times \mathrm{m}^2 + 3000 \times \mathrm{banheiros} - 1500 \times \mathrm{distância~do~centro~km}$$

## Métricas de Desempenho de uma Regressão

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/erro-reg.png?raw=1" alt="erro-reg" style="width: 400px;"/>

### *Mean Absolute Error* (MAE) - Erro Absoluto Médio

O erro médio absoluto, MAE (da sigla em inglês Mean Absolute Error), é calculado a partir da **média dos erros absolutos**, ou seja, utilizamos o módulo de cada erro para evitar a subestimação, isso porque, o valor é menos afetado por pontos especialmente extremos (outliers).

Cada erro, pode ser interpretado como a diferença entre Y e Ŷ e assim, temos:

$$MAE = \frac{1}{m}\Sigma_{i=1}^{m}{|\hat{y}_i - y_i|}$$

Utilizamos essa medida em séries temporais, pois há casos em que o erro negativo pode zerar o positivo ou dar uma ideia de que o modelo é preciso. Mas aqui, medimos apenas a distância do valor real, independente de ser acima ou abaixo.

### *Mean Squared Error* (MSE) - Erro Quadrático Médio

O erro quadrático médio, MSE (da sigla em inglês Mean Squared Error), é comumente usado para verificar a **acurácia de modelos** e dá um maior peso aos maiores erros, já que, ao ser calculado, cada erro é elevado ao quadrado individualmente e, após isso, a média desses erros quadráticos é calculada.

Usando o mesmo conceito de erro utilizado anteriormente, temos a equação abaixo:

$$MSE = \frac{1}{m}\Sigma_{i=1}^{m}{(\hat{y}_i - y_i)^2}$$

Por conta do expoente ao quadrado que o erro assume, essa métrica é bastante sensível a outliers (valores discrepantes) e, caso tenha muitos erros significativos em sua análise, essa métrica poderá ser extrapolada.

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/gradient-descent.gif?raw=1" alt="gradient-descent-animation" style="width: 60%;"/>

<img src="https://github.com/bilhalvadiego/ds-course/blob/main/notebooks/images/gradient-descent-2.gif?raw=1" alt="gradient-descent-animation" style="width: 60%;"/>

## Exemplo com o dataset [Boston House Prices](https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset)


* $N = 506$
* Atributos: 13
    * `CRIM` crime per capita da região
    * `ZN` proporção de terra residencial
    * `INDUS` proporção terra comercial não-varejista
    * `CHAS` *Dummy* se fica as margens do Charles River (1 ou 0)
    * `NOX` concentração de óxido nítrico (partes por 10 milhões)
    * `RM` número de quartos
    * `AGE` idade da residência
    * `DIS` distância dos cinco centros de emprego de Boston
    * `RAD` acessibilidade às rodovias radiais
    * `TAX` valor do IPTU por 10,000 USD
    * `PTRATIO` relação professor-aluno (*pupil-teacher ratio*) da região
    * `B` proporção de afro-descendentes na região
    * `LSTAT` porcentagem de população de baixa-renda
* **Variável resposta**: valor da casa por 10,000 USD

In [1]:
import pandas as pd
SEED = 5252

boston = pd.read_csv('https://raw.githubusercontent.com/bilhalvadiego/ds-course/main/notebooks/data/BostonHousing.csv')


In [2]:
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
X = boston.loc[:,'crim':'lstat']
y = boston['medv']

In [4]:
print(f'Variáveis preditoras: {X.columns}')
print(f'Variável Predita: {y.name}')

Variáveis preditoras: Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat'],
      dtype='object')
Variável Predita: medv


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_norm = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [6]:
X_norm.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


In [7]:
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

Tamanho de X: (506, 13)
Tamanho de y: (506,)


### Quebrando dataset em `train` e `test`

Usar a função do Scikit-Learn [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### Argumentos:

* matriz a ser dividida - `X` ou `y`
* `test_size` - `float` ou `int` do tamanho do dataset de teste (padrão $0.25$)
* `train_size` - padrão `1 - test_size`
* `random_state` - `int` - seed do gerador de número randômicos (replicabilidade)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_norm,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=SEED)

In [32]:
print(f"Tamanho de X_train: {X_train.shape}")
print(f"Tamanho de X_test: {X_test.shape}")
print(f"Tamanho de y_train: {y_train.shape}")
print(f"Tamanho de y_test: {y_test.shape}")

Tamanho de X_train: (379, 13)
Tamanho de X_test: (127, 13)
Tamanho de y_train: (379,)
Tamanho de y_test: (127,)


### Regressão Linear
Usar o estimador do Scikit-Learn [`sklearn.linear_model.LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

#### Retorna:
* Objeto `estimator` do Scikit-Learn

In [33]:
from sklearn.linear_model import LinearRegression

clf = LinearRegression()

### Classe `Estimators`

* `.fit()` - Treina o Modelo
    * `X`
    * `y`
* `.predict()` - Gera predições do modelo
    * `X`
* `.coef_` - Retorna os coeficientes do modelo ($\theta_i$)
* `.intercept_` - Retorna o viés/constante (*bias/intercept*) do modelo ($\theta_0$)

In [35]:
clf.fit(X_train, y_train)

LinearRegression()

In [38]:
lista = clf.coef_.tolist()


# ŷ = clf.intercept_ + lista[0]*CRIM + lista[1]*ZN...+lista[12]*LSTAT

[-0.9185017510720556,
 1.1914025471845424,
 -0.012463897361086107,
 0.648948261956646,
 -1.889661400695244,
 2.53174277589179,
 -0.07222152586176533,
 -3.162801161473702,
 2.680366669144891,
 -2.083794009973667,
 -1.7986775574248735,
 1.0042866891727604,
 -3.767893645060866]

In [39]:
# Coeficientes do modelo
for feature, coef in zip(boston.columns.tolist(), clf.coef_.tolist()):
    print(f"{feature}: {round(coef, 2)}")

# Constante do modelo
print(f"Constante: {round(clf.intercept_, 2)}")

crim: -0.92
zn: 1.19
indus: -0.01
chas: 0.65
nox: -1.89
rm: 2.53
age: -0.07
dis: -3.16
rad: 2.68
tax: -2.08
ptratio: -1.8
b: 1.0
lstat: -3.77
Constante: 22.56


In [13]:
# ŷ = 34.69 -.11*crim + 0.05*zn - 0.0 * indus + 2.56*chas...-.53*lstat
# ŷ = 34.69 -.11*.0232 + 0.05*0 - 0 + 2.56*1...-.53*2.34
# ŷ = 22

### Erro do Modelo


In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = clf.predict(X_test)

print(f"MSE de Teste: {mean_squared_error(y_test, y_pred):1.1f}")
print(f"MAE de Teste: {mean_absolute_error(y_test, y_pred):1.1f}")

MSE de Teste: 19.9
MAE de Teste: 3.1


In [41]:
from sklearn.metrics import r2_score

r_quadrado = r2_score(y_test, y_pred)

print('O R² é de {0}. Isto é, {1}% dos casos são explicados pelo modelo.'.format(round(r_quadrado,3), round(r_quadrado*100,1)))

O R² é de 0.786. Isto é, 78.6% dos casos são explicados pelo modelo.


## Atividade - Regressão com o dataset [Diabetes](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset)

* $N = 442$
* Atributos: 10
    * `age`
    * `sex`
    * `bmi` Índice de Massa Corpórea (IMC) - *Body Mass Index* (BMI)
    * `bp` pressão arterial média *blood pressure* (bp)
    * `s1` colesterol total
    * `s2` colesterol LDL
    * `s3` colesterol HDL
    * `s4` colesterol VLDL
    * `s5` triglicerides
    * `s6` glicose
* Variável dependente: medida quantitativa de progressão da diabetes

* Rodem o `LinearRegression()` nos dados de treino e mensure o desempenho nos dados de teste.

>Obs: usar `test_size = 0.25` e `random_state = 123`

In [42]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = diabetes['data']
y = diabetes['target']

In [43]:
print(f"Nomes dos Atributos: {diabetes['feature_names']}")
print(f"Tamanho de X: {X.shape}")
print(f"Tamanho de y: {y.shape}")

Nomes dos Atributos: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
Tamanho de X: (442, 10)
Tamanho de y: (442,)


In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=18)

In [19]:
print(f"Tamanho de X_train: {X_train.shape}")
print(f"Tamanho de X_test: {X_test.shape}")
print(f"Tamanho de y_train: {y_train.shape}")
print(f"Tamanho de y_test: {y_test.shape}")

Tamanho de X_train: (331, 10)
Tamanho de X_test: (111, 10)
Tamanho de y_train: (331,)
Tamanho de y_test: (111,)


In [20]:
from sklearn.linear_model import LinearRegression

In [21]:
clf = LinearRegression()

In [22]:
clf.fit(X_train, y_train)

LinearRegression()

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = clf.predict(X_test)

print(f"MSE de Teste: {mean_squared_error(y_test, y_pred):1.1f}")
print(f"MAE de Teste: {mean_absolute_error(y_test, y_pred):1.1f}")

MSE de Teste: 3006.1
MAE de Teste: 47.0


In [24]:
from sklearn.metrics import r2_score

r_quadrado = r2_score(y_test, y_pred)

print('O R² é de {0}. Isto é, {1}% dos casos são explicados pelo modelo.'.format(round(r_quadrado,3), round(r_quadrado*100,1)))

O R² é de 0.506. Isto é, 50.6% dos casos são explicados pelo modelo.


In [25]:
base_origin = pd.DataFrame(X.copy(), columns=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'])

In [26]:
base_origin['target'] = y

In [27]:
base_origin.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [28]:
base_origin.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-2.511817e-19,1.230790e-17,-2.245564e-16,-4.797570e-17,-1.381499e-17,3.918434e-17,-5.777179e-18,-9.042540e-18,9.293722e-17,1.130318e-17,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123988e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260971e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665608e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324559e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670422e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947171e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564379e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243232e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320436e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335973e-01,1.356118e-01,346.000000
